In [111]:
from importlib import reload
import perplexity
import lda_neuron_selection
reload(lda_neuron_selection)
reload(perplexity)
from perplexity import PerplexityAnalyzer
from transformers import GPT2Tokenizer
from gpt2 import GPT2LMHeadModel
import pandas as pd
from neuron_selection import NeuronSelector
from lda_neuron_selection import LdaNeuronSelector
import numpy as np
import torch

from matplotlib import pyplot as plt, rcParams
from matplotlib_inline.backend_inline import set_matplotlib_formats

rcParams["figure.figsize"] = (8, 5)
rcParams["figure.dpi"] = 100
set_matplotlib_formats("retina")
plt.style.use("seaborn")

In [65]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
model.eval()

perplexity_df = pd.read_csv("data/wiki.csv")

In [94]:
neuron_selector = NeuronSelector()
neurons_per_layer = neuron_selector.get_neurons_per_layer(n_neurons=100, percentile=0.8, method="pca_correlation")

In [8]:
neuron_selector = LdaNeuronSelector(filename="output/output.npz")
neurons_per_layer = neuron_selector.get_lda_neurons_per_layer(n_layers=25, percentile=0.8, method="correlation")

AttributeError: 'LdaNeuronSelector' object has no attribute 'get_neurons_per_layer'

In [95]:
perp_analyzer = PerplexityAnalyzer(model, tokenizer, df=perplexity_df.sample(10))

In [97]:
perp_analyzer.get_average_perplexity(neurons_per_layer=neurons_per_layer, force_emotion="negative", force_with="transform")

3812.784415879751

In [67]:
BATCH_SIZE = 5
n_batches = len(perplexity_df) // BATCH_SIZE + 1
batch = next(iter(np.array_split(perplexity_df, n_batches)))
sentences = batch["sentence"].values.tolist()
sentences

["It tells the story of the making of Bandolier in the band's own words, including an extensive audio interview with Burke Shelley.",
 'Under his conditions it produces pitchers that are almost completley black.',
 "Given that they're pulling out all the stops, they deserve all the acclaim that surrounded them for their first two albums.",
 'It was their second EP and, like their first EP, was released only in Australia.',
 'Contains a detailed monograph on Schisandra chinensis as well as highlights health benefits.']

In [69]:
tokenizer.pad_token = tokenizer.eos_token
input = tokenizer(sentences, padding=True, return_tensors="pt")

In [70]:
input['input_ids'].shape

torch.Size([5, 27])

In [71]:
with torch.no_grad():
    output = model(
        **input,
        neurons_per_layer=neurons_per_layer,
        force_emotion="negative",
        force_with="per_neuron",
    )

In [73]:
log_probs = torch.nn.functional.log_softmax(output.logits, dim=2)
# likelihood_sequence = self.get_likelihood_sequence(input, log_probs)

In [79]:
[
    log_probs[:, i, token_index].item()
    for i, token_index in enumerate(input.input_ids[:, 1:])
]

ValueError: only one element tensors can be converted to Python scalars

In [89]:
token_ids = next(iter(input.input_ids[:, 1:].T))
np.diag(log_probs[:, 0, token_ids])

array([-7.7975893, -5.226864 , -1.6991314, -2.449772 , -4.5495253],
      dtype=float32)

In [88]:
token_ids

tensor([4952,  465,  326,  373, 1299])

In [98]:
results = [
    {'a': 1, 'b': 2},
    {'a': 3, 'b': 4},
]

pd.DataFrame(results)

,a,b
0,1,2
1,3,4


In [112]:
lda_neuron_selector = LdaNeuronSelector(filename="output/output.npz")

In [120]:
neurons_per_layer = lda_neuron_selector.get_lda_neurons_per_layer(
    n_layers=25,
    percentile=0.8,
    method="logistic_regression",
)

In [126]:
hyperparams = [{
    "selection_method": [
        "correlation",
        "logistic_regression",
        "pca_correlation",
        "pca_log_reg",
    ],
    "n_neurons": [100, 200, 500, 1000, 5000],
    "percentile": [0.5, 0.8, 1],
}]

In [127]:
# Import parameter grid
from sklearn.model_selection import ParameterGrid

for params in ParameterGrid(hyperparams):
    print(params)

{'n_neurons': 100, 'percentile': 0.5, 'selection_method': 'correlation'}
{'n_neurons': 100, 'percentile': 0.5, 'selection_method': 'logistic_regression'}
{'n_neurons': 100, 'percentile': 0.5, 'selection_method': 'pca_correlation'}
{'n_neurons': 100, 'percentile': 0.5, 'selection_method': 'pca_log_reg'}
{'n_neurons': 100, 'percentile': 0.8, 'selection_method': 'correlation'}
{'n_neurons': 100, 'percentile': 0.8, 'selection_method': 'logistic_regression'}
{'n_neurons': 100, 'percentile': 0.8, 'selection_method': 'pca_correlation'}
{'n_neurons': 100, 'percentile': 0.8, 'selection_method': 'pca_log_reg'}
{'n_neurons': 100, 'percentile': 1, 'selection_method': 'correlation'}
{'n_neurons': 100, 'percentile': 1, 'selection_method': 'logistic_regression'}
{'n_neurons': 100, 'percentile': 1, 'selection_method': 'pca_correlation'}
{'n_neurons': 100, 'percentile': 1, 'selection_method': 'pca_log_reg'}
{'n_neurons': 200, 'percentile': 0.5, 'selection_method': 'correlation'}
{'n_neurons': 200, 'per